In [29]:
# http://bloggerkaiweng.blogspot.tw/2016/09/python_13.html

import pandas as pd

def getstocklist(stksite):
    df = pd.read_html(stksite, encoding='big5hkscs', header=0) # header=0 - 取消index
    newdf = df[0][df[0]['產業別'] > '0'] # 取得股票資料(排除權證等) - 其他證券無產業別
    
    del newdf['國際證券辨識號碼(ISIN Code)'], newdf['CFICode'], newdf['備註'] # 刪除不要的欄位
    newdf_no = newdf['有價證券代號及名稱'].str.split(' ', expand=True) # 迭代處理所有資料
    newdf_no = newdf_no.reset_index(drop=True)
    newdf = newdf.reset_index(drop=True)
    
    for i in newdf_no.index: #字串處理
        if '　' in newdf_no.iat[i,0]:
            newdf_no.iat[i,1] = newdf_no.iat[i,0].split('　')[1]
            newdf_no.iat[i,0] = newdf_no.iat[i,0].split('　')[0]
    
    newdf=newdf_no.join(newdf)
    newdf=newdf.rename(columns = {0:'ticker',1:'股票名稱'})
    del newdf['有價證券代號及名稱']
    return newdf

newdf1 = getstocklist('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2') #爬上市
newdf2 = getstocklist('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4') #爬上櫃

newdfok = pd.concat([newdf1,newdf2],keys='ticker') #合併上市櫃資料
newdfok = newdfok.sort_values(by='ticker') #依股票代碼排序

final_df = (newdfok['ticker']).reset_index()
del final_df['level_0'], final_df['level_1']

#final_df.to_csv('stocklist-E.csv')
print('done!')

done!
